<a href="https://colab.research.google.com/github/aniroxxsc/Neural-Machine-Translation-System/blob/main/IITB_Corpus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import string
import re
from string import digits
import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model
import pickle


In [ ]:
from google.colab import drive
drive.mount('/content/Drive')

Drive already mounted at /content/Drive; to attempt to forcibly remount, call drive.mount("/content/Drive", force_remount=True).


In [ ]:
os.listdir('/content/Drive/My Drive/Application of ML in industries/')

['UCI_Credit_Card.csv',
 'indian_liver_patient.csv',
 'winequality-red.csv',
 'Lab2 Housing price prediction',
 'Lab4',
 'NMT corpus IITB']

In [ ]:
engl=[]
file=open('/content/Drive/My Drive/Application of ML in industries/NMT corpus IITB/IITB.en-hi.en', 'r')
for row in file:
  engl.append(row)

In [ ]:
len(engl)

1609682

In [ ]:
hinl=[]
file=open('/content/Drive/My Drive/Application of ML in industries/NMT corpus IITB/IITB.en-hi.hi', 'r')
for row in file:
  hinl.append(row)

In [ ]:
len(hinl)

1609682

In [ ]:
eng_try = pd.DataFrame(engl, columns=['eng'])
hin_try = pd.DataFrame(hinl, columns=['hin'])

In [ ]:
eng_try

,eng
0,Give your application an accessibility workout\n
1,Accerciser Accessibility Explorer\n
2,The default plugin layout for the bottom panel\n
3,The default plugin layout for the top panel\n
4,A list of plugins that are disabled by default\n
...,...
1609677,The programme will be streamed live via:\n
1609678,Ministry of Education Facebook Page: https://w...
1609679,"UGC YouTube Channel, PIB YouTube Channel,\n"
1609680,UGC Twitter Handle (@ugc_india) : https://twit...


In [ ]:
df = eng_try
df['hin'] = hin_try['hin'] 

In [ ]:
df.isnull().sum()

eng    0
hin    0
dtype: int64

In [ ]:
#lower casing all entries
df['eng'] = df['eng'].str.lower()
df['hin'] = df['hin'].str.lower()

In [ ]:
#removing quotes from dataset
df['eng'] = df['eng'].str.replace('"','')
df['eng'] = df['eng'].str.replace("'",'')
df['hin'] = df['hin'].str.replace('"','')
df['hin'] = df['hin'].str.replace("'",'')

In [ ]:
#remove all special characters
exclude = set(string.punctuation)
len(exclude)
for p in exclude:
    df['eng'] = df['eng'].str.replace(p,'')
    df['hin'] = df['hin'].str.replace(p,'')

In [ ]:
remove_digits = str.maketrans('', '', digits)
df['eng']=df['eng'].apply(lambda x: x.translate(remove_digits))
df['hin']=df['hin'].apply(lambda x: x.translate(remove_digits))

df['hin'] = df['hin'].apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))

# Remove extra spaces
df['eng']=df['eng'].apply(lambda x: x.strip())
df['hin']=df['hin'].apply(lambda x: x.strip())
df['eng']=df['eng'].apply(lambda x: re.sub(" +", " ", x))
df['hin']=df['hin'].apply(lambda x: re.sub(" +", " ", x))

In [ ]:
df

,eng,hin
0,give your application an accessibility workout,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें
1,accerciser accessibility explorer,एक्सेर्साइसर पहुंचनीयता अन्वेषक
2,the default plugin layout for the bottom panel,निचले पटल के लिए डिफोल्ट प्लगइन खाका
3,the default plugin layout for the top panel,ऊपरी पटल के लिए डिफोल्ट प्लगइन खाका
4,a list of plugins that are disabled by default,उन प्लगइनों की सूची जिन्हें डिफोल्ट रूप से निष...
...,...,...
1609677,the programme will be streamed live via,कार्यक्रम को इनके जरिए लाइव स्ट्रीम किया जाएगा
1609678,ministry of education facebook page httpswwwfa...,मानव संसाधन विकास मंत्रालय का फेसबुक पेज https...
1609679,ugc youtube channel pib youtube channel,यूजीसी यूट्यूब चैनल पीआईबी यूट्यूब चैनल
1609680,ugc twitter handle ugcindia httpstwittercomugc...,यूजीसी ट्विटर हैंडल ugcindia httpstwittercomug...


In [ ]:
df['length_eng_sentence']=df['eng'].apply(lambda x:len(x.split(" ")))
df['length_hin_sentence']=df['hin'].apply(lambda x:len(x.split(" ")))
df.head()

,eng,hin,length_eng_sentence,length_hin_sentence
0,give your application an accessibility workout,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें,6,8
1,accerciser accessibility explorer,एक्सेर्साइसर पहुंचनीयता अन्वेषक,3,3
2,the default plugin layout for the bottom panel,निचले पटल के लिए डिफोल्ट प्लगइन खाका,8,7
3,the default plugin layout for the top panel,ऊपरी पटल के लिए डिफोल्ट प्लगइन खाका,8,7
4,a list of plugins that are disabled by default,उन प्लगइनों की सूची जिन्हें डिफोल्ट रूप से निष...,9,12


In [ ]:
print("maximum length of Hindi Sentence ",max(df['length_hin_sentence']))
print("maximum length of English Sentence ",max(df['length_eng_sentence']))

maximum length of Hindi Sentence  1136
maximum length of English Sentence  1452


In [ ]:
df

,eng,hin,length_eng_sentence,length_hin_sentence
0,give your application an accessibility workout,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें,6,8
1,accerciser accessibility explorer,एक्सेर्साइसर पहुंचनीयता अन्वेषक,3,3
2,the default plugin layout for the bottom panel,निचले पटल के लिए डिफोल्ट प्लगइन खाका,8,7
3,the default plugin layout for the top panel,ऊपरी पटल के लिए डिफोल्ट प्लगइन खाका,8,7
4,a list of plugins that are disabled by default,उन प्लगइनों की सूची जिन्हें डिफोल्ट रूप से निष...,9,12
...,...,...,...,...
1609677,the programme will be streamed live via,कार्यक्रम को इनके जरिए लाइव स्ट्रीम किया जाएगा,7,8
1609678,ministry of education facebook page httpswwwfa...,मानव संसाधन विकास मंत्रालय का फेसबुक पेज https...,6,8
1609679,ugc youtube channel pib youtube channel,यूजीसी यूट्यूब चैनल पीआईबी यूट्यूब चैनल,6,6
1609680,ugc twitter handle ugcindia httpstwittercomugc...,यूजीसी ट्विटर हैंडल ugcindia httpstwittercomug...,5,5


In [ ]:
alphabets =['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

In [ ]:
df1=df.copy()

In [ ]:
df1.head()

,eng,hin,length_eng_sentence,length_hin_sentence
0,give your application an accessibility workout,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें,6,8
1,accerciser accessibility explorer,एक्सेर्साइसर पहुंचनीयता अन्वेषक,3,3
2,the default plugin layout for the bottom panel,निचले पटल के लिए डिफोल्ट प्लगइन खाका,8,7
3,the default plugin layout for the top panel,ऊपरी पटल के लिए डिफोल्ट प्लगइन खाका,8,7
4,a list of plugins that are disabled by default,उन प्लगइनों की सूची जिन्हें डिफोल्ट रूप से निष...,9,12


In [ ]:
df2 =df[0:100].copy()

In [ ]:
df2.head(35)

,eng,hin,length_eng_sentence,length_hin_sentence
0,give your application an accessibility workout,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें,6,8
1,accerciser accessibility explorer,एक्सेर्साइसर पहुंचनीयता अन्वेषक,3,3
2,the default plugin layout for the bottom panel,निचले पटल के लिए डिफोल्ट प्लगइन खाका,8,7
3,the default plugin layout for the top panel,ऊपरी पटल के लिए डिफोल्ट प्लगइन खाका,8,7
4,a list of plugins that are disabled by default,उन प्लगइनों की सूची जिन्हें डिफोल्ट रूप से निष...,9,12
5,highlight duration,अवधि को हाइलाइट रकें,2,4
6,the duration of the highlight box when selecti...,पहुंचनीय आसंधि नोड को चुनते समय हाइलाइट बक्से ...,10,10
7,highlight border color,सीमांत बोर्डर के रंग को हाइलाइट करें,3,7
8,the color and opacity of the highlight border,हाइलाइट किए गए सीमांत का रंग और अपारदर्शिता।,8,8
9,highlight fill color,भराई के रंग को हाइलाइट करें,3,6


In [ ]:

def f(string):
  for i in string.split():
    for j in i:
      if j in alphabets:
        return '-1'
  return string

In [ ]:
#First trying on df2, for checking if function f is working fine 
#df2['hin'] = df2['hin'].apply(filter(df2))
df2.drop(0,inplace=True)

In [ ]:
df2['hin'] = df2['hin'].apply(lambda x: f(x), )

In [ ]:
df2.head(35)

,eng,hin,length_eng_sentence,length_hin_sentence
1,accerciser accessibility explorer,एक्सेर्साइसर पहुंचनीयता अन्वेषक,3,3
2,the default plugin layout for the bottom panel,निचले पटल के लिए डिफोल्ट प्लगइन खाका,8,7
3,the default plugin layout for the top panel,ऊपरी पटल के लिए डिफोल्ट प्लगइन खाका,8,7
4,a list of plugins that are disabled by default,उन प्लगइनों की सूची जिन्हें डिफोल्ट रूप से निष...,9,12
5,highlight duration,अवधि को हाइलाइट रकें,2,4
6,the duration of the highlight box when selecti...,पहुंचनीय आसंधि नोड को चुनते समय हाइलाइट बक्से ...,10,10
7,highlight border color,सीमांत बोर्डर के रंग को हाइलाइट करें,3,7
8,the color and opacity of the highlight border,हाइलाइट किए गए सीमांत का रंग और अपारदर्शिता।,8,8
9,highlight fill color,भराई के रंग को हाइलाइट करें,3,6
10,the color and opacity of the highlight fill,हाइलाइट किया गया भराई का रंग और पारदर्शिता।,8,8


In [ ]:
df1.head(35)

,eng,hin,length_eng_sentence,length_hin_sentence
0,give your application an accessibility workout,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें,6,8
1,accerciser accessibility explorer,एक्सेर्साइसर पहुंचनीयता अन्वेषक,3,3
2,the default plugin layout for the bottom panel,निचले पटल के लिए डिफोल्ट प्लगइन खाका,8,7
3,the default plugin layout for the top panel,ऊपरी पटल के लिए डिफोल्ट प्लगइन खाका,8,7
4,a list of plugins that are disabled by default,उन प्लगइनों की सूची जिन्हें डिफोल्ट रूप से निष...,9,12
5,highlight duration,अवधि को हाइलाइट रकें,2,4
6,the duration of the highlight box when selecti...,पहुंचनीय आसंधि नोड को चुनते समय हाइलाइट बक्से ...,10,10
7,highlight border color,सीमांत बोर्डर के रंग को हाइलाइट करें,3,7
8,the color and opacity of the highlight border,हाइलाइट किए गए सीमांत का रंग और अपारदर्शिता।,8,8
9,highlight fill color,भराई के रंग को हाइलाइट करें,3,6


In [ ]:
df1['hin'] = df1['hin'].apply(lambda x: f(x))

In [ ]:
df1.head(35)

,eng,hin,length_eng_sentence,length_hin_sentence
0,give your application an accessibility workout,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें,6,8
1,accerciser accessibility explorer,एक्सेर्साइसर पहुंचनीयता अन्वेषक,3,3
2,the default plugin layout for the bottom panel,निचले पटल के लिए डिफोल्ट प्लगइन खाका,8,7
3,the default plugin layout for the top panel,ऊपरी पटल के लिए डिफोल्ट प्लगइन खाका,8,7
4,a list of plugins that are disabled by default,उन प्लगइनों की सूची जिन्हें डिफोल्ट रूप से निष...,9,12
5,highlight duration,अवधि को हाइलाइट रकें,2,4
6,the duration of the highlight box when selecti...,पहुंचनीय आसंधि नोड को चुनते समय हाइलाइट बक्से ...,10,10
7,highlight border color,सीमांत बोर्डर के रंग को हाइलाइट करें,3,7
8,the color and opacity of the highlight border,हाइलाइट किए गए सीमांत का रंग और अपारदर्शिता।,8,8
9,highlight fill color,भराई के रंग को हाइलाइट करें,3,6


In [ ]:
df1['hin'][20]

'-1'

In [ ]:
len(df1)

1609682

In [ ]:
df

,eng,hin,length_eng_sentence,length_hin_sentence
0,give your application an accessibility workout,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें,6,8
1,accerciser accessibility explorer,एक्सेर्साइसर पहुंचनीयता अन्वेषक,3,3
2,the default plugin layout for the bottom panel,निचले पटल के लिए डिफोल्ट प्लगइन खाका,8,7
3,the default plugin layout for the top panel,ऊपरी पटल के लिए डिफोल्ट प्लगइन खाका,8,7
4,a list of plugins that are disabled by default,उन प्लगइनों की सूची जिन्हें डिफोल्ट रूप से निष...,9,12
...,...,...,...,...
1609677,the programme will be streamed live via,कार्यक्रम को इनके जरिए लाइव स्ट्रीम किया जाएगा,7,8
1609678,ministry of education facebook page httpswwwfa...,मानव संसाधन विकास मंत्रालय का फेसबुक पेज https...,6,8
1609679,ugc youtube channel pib youtube channel,यूजीसी यूट्यूब चैनल पीआईबी यूट्यूब चैनल,6,6
1609680,ugc twitter handle ugcindia httpstwittercomugc...,यूजीसी ट्विटर हैंडल ugcindia httpstwittercomug...,5,5


In [ ]:
l=[]
for i in range(0,len(df)):
  if df1['hin'][i]=="-1":
    l.append(i)
df1.drop(l,inplace=True)

In [ ]:
df1.head(35)

,eng,hin,length_eng_sentence,length_hin_sentence
0,give your application an accessibility workout,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें,6,8
1,accerciser accessibility explorer,एक्सेर्साइसर पहुंचनीयता अन्वेषक,3,3
2,the default plugin layout for the bottom panel,निचले पटल के लिए डिफोल्ट प्लगइन खाका,8,7
3,the default plugin layout for the top panel,ऊपरी पटल के लिए डिफोल्ट प्लगइन खाका,8,7
4,a list of plugins that are disabled by default,उन प्लगइनों की सूची जिन्हें डिफोल्ट रूप से निष...,9,12
5,highlight duration,अवधि को हाइलाइट रकें,2,4
6,the duration of the highlight box when selecti...,पहुंचनीय आसंधि नोड को चुनते समय हाइलाइट बक्से ...,10,10
7,highlight border color,सीमांत बोर्डर के रंग को हाइलाइट करें,3,7
8,the color and opacity of the highlight border,हाइलाइट किए गए सीमांत का रंग और अपारदर्शिता।,8,8
9,highlight fill color,भराई के रंग को हाइलाइट करें,3,6


In [ ]:
df1.shape

(1478314, 4)

In [ ]:
df1['length_eng_sentence']=df1['eng'].apply(lambda x:len(x.split(" ")))
df1['length_hin_sentence']=df1['hin'].apply(lambda x:len(x.split(" ")))
df1.head()

,eng,hin,length_eng_sentence,length_hin_sentence
0,give your application an accessibility workout,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें,6,8
1,accerciser accessibility explorer,एक्सेर्साइसर पहुंचनीयता अन्वेषक,3,3
2,the default plugin layout for the bottom panel,निचले पटल के लिए डिफोल्ट प्लगइन खाका,8,7
3,the default plugin layout for the top panel,ऊपरी पटल के लिए डिफोल्ट प्लगइन खाका,8,7
4,a list of plugins that are disabled by default,उन प्लगइनों की सूची जिन्हें डिफोल्ट रूप से निष...,9,12


In [ ]:
# Add start and end tokens to target sequences
df1['hin'] = df1['hin'].apply(lambda x : 'START_ '+ x + ' _END')

In [ ]:
#df1 = df1[:25000]

In [ ]:
df1

,eng,hin,length_eng_sentence,length_hin_sentence
0,give your application an accessibility workout,START_ अपने अनुप्रयोग को पहुंचनीयता व्यायाम का...,6,8
1,accerciser accessibility explorer,START_ एक्सेर्साइसर पहुंचनीयता अन्वेषक _END,3,3
2,the default plugin layout for the bottom panel,START_ निचले पटल के लिए डिफोल्ट प्लगइन खाका _END,8,7
3,the default plugin layout for the top panel,START_ ऊपरी पटल के लिए डिफोल्ट प्लगइन खाका _END,8,7
4,a list of plugins that are disabled by default,START_ उन प्लगइनों की सूची जिन्हें डिफोल्ट रूप...,9,12
...,...,...,...,...
1609675,a number of dignitaries including the chairman...,START_ कई गणमान्य व्यक्ति राष्ट्रीय शिक्षा नीत...,28,32
1609676,vicechancellors of universities directors of i...,START_ विश्वविद्यालयों के कुलपति संस्थानों के ...,18,18
1609677,the programme will be streamed live via,START_ कार्यक्रम को इनके जरिए लाइव स्ट्रीम किय...,7,8
1609679,ugc youtube channel pib youtube channel,START_ यूजीसी यूट्यूब चैनल पीआईबी यूट्यूब चैनल...,6,6


In [ ]:
df1[df1['length_eng_sentence']>30].shape

(141078, 4)

In [ ]:
df1 =df1[df1['length_eng_sentence']<20]
df1 =df1[df1['length_hin_sentence']<20]
print(df1.shape)

(1072386, 4)


Remove hinctuations from hindi text

In [ ]:
hind_punc = ['”','“',]
for p in hind_punc:
    df1['hin'] = df1['hin'].str.replace(p,'')

In [ ]:
df1.tail()

,eng,hin,length_eng_sentence,length_hin_sentence
1609672,the conclave is being organised by the ministr...,START_ इस सम्‍मेलन का आयोजन मानव संसाधन विकास ...,14,17
1609676,vicechancellors of universities directors of i...,START_ विश्वविद्यालयों के कुलपति संस्थानों के ...,18,18
1609677,the programme will be streamed live via,START_ कार्यक्रम को इनके जरिए लाइव स्ट्रीम किय...,7,8
1609679,ugc youtube channel pib youtube channel,START_ यूजीसी यूट्यूब चैनल पीआईबी यूट्यूब चैनल...,6,6
1609681,it would also be broadcast on dd news,START_ कार्यक्रम को डीडी न्यूज पर भी प्रसारित ...,8,9


Many rows translation is not correct, msotly includes the starting rows or the ending rows in dataset df1 when sorted w.r.t 'hin' column

In [ ]:
df1 = df1.sort_values(by=['hin'])
df1[-2000:-900]

,eng,hin,length_eng_sentence,length_hin_sentence
473295,hogenakkal falls,START_ होगेन्नाकल _END,2,1
1265616,hojagiri dance,START_ होजगिरि नृत्य _END,2,2
921479,hosiery,START_ होजरी _END,1,1
1202749,hosiery woolen garments,START_ होजरी एवं ऊनी परिधान _END,3,4
586587,josé acasuso,START_ होज़े अकासुसो _END,2,2
...,...,...,...,...
1190387,administration of this act is the ministry of ...,START_ ै। संचार और सूचना प्रौद्योगिकी मंत्रालय...,13,12
653594,books,START_ ो किताबें _END,1,2
452796,a books,START_ ो किताबें _END,2,2
1096904,books,START_ ो किताबें _END,1,2


Have to skip 13843 rows from starting

have to skip *2000*
 rows from end

In [ ]:
df1 = df1[13562:]
df1 = df1[:-2000]

In [ ]:
df1[200:300]

,eng,hin,length_eng_sentence,length_hin_sentence
582784,in theory india has a national grid in practic...,START_ ं सिद्धांत में देश में एक राष्ट्रौईय ग्...,14,18
693937,mmr contains three separate vaccines in one in...,START_ ंंएम एम आर में तीन अलग अलग टीके होते है...,8,18
478316,mmr can prevent these diseases in a combined i...,START_ ंंएम एम आर यह रोग एक ही सुऋ से मिटा सकत...,9,12
568453,mmr protects your child and your family agains...,START_ ंंष् आप को और आप के परिवार को मेअस्लेस्...,12,16
530613,mumps was the leading cause of viral meningiti...,START_ ंंष् के टीके उपलब्ध होने से पहले बच्चों...,16,16
...,...,...,...,...
158270,loading a,START_ अ लोड किया जा रहा है _END,2,6
1070743,the square root of a squared plus b squared,START_ अ वर्ग प्लस बी वर्ग का वर्गमूल। _END,9,7
158115,a vs b overview,START_ अ वि ब ओवरव्यू _END,4,4
158116,a vs c overview,START_ अ वि स ओवरव्यू _END,4,4


In [ ]:
df1.shape

(1056824, 4)

In [ ]:
### Get English and Hindi Vocabulary
all_hindi_words=set()
for hin in df1['hin']:
    for word in hin.split():
        if word not in all_hindi_words:
            all_hindi_words.add(word)

all_eng_words=set()
for eng in df1['eng']:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)
print(len(all_eng_words))
print(len(all_hindi_words))

130684
236639


In [ ]:
max_length_src=max(df1['length_hin_sentence'])
max_length_tar=max(df1['length_eng_sentence'])

In [ ]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_hindi_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_hindi_words)
num_encoder_tokens, num_decoder_tokens

(130684, 236639)

In [ ]:
tar = pd.DataFrame(target_words)
l = []
tar[100:120]
for i in range(0,323):
  l.append(tar[0][i])

In [ ]:
tar[:50]

,0
0,START_
1,_END
2,£
3,¥
4,¨
5,¨éé
6,¨निश्चय
7,©
8,°
9,°°


In [ ]:
for i in range(236612	,236638):
  l.append(tar[0][i])

In [ ]:
tar[-50::]

,0
236589,’को
236590,’गुरुशिष्य
236591,’तो
236592,’पट्टी’
236593,’पर
236594,’पश्चीयन
236595,’पूछताछ
236596,’प्रक्षेपक’
236597,’प्राप्त
236598,’प्रेक्षक’


In [ ]:
l

['START_',
 '_END',
 '£',
 '¥',
 '¨',
 '¨éé',
 '¨निश्चय',
 '©',
 '°',
 '°°',
 '°से',
 '°सेल्सियस',
 '²',
 '³',
 '´éšééç',
 'º',
 'º।',
 '¼',
 '¼½',
 '½',
 '½éþ।',
 '½θώ।',
 '½ιώ',
 '½ιώ।',
 '¾',
 '×',
 'à¤',
 'à¤\x86à¤\x87à¤\x95à¥\x89à¤¨à¥\x8dà¤¸',
 'à¤\x86à¤\x95à¤¾à¤°',
 'à¤\x86à¤«',
 'à¤\x86à¤°à¥\x87à¤\x96à¤¿à¤¤',
 'à¤\x86à¤µà¤¾à¤\x9c',
 'à¤\x87à¤¸',
 'à¤\x89',
 'à¤\x89à¤¤à¥\x8dà¤¤à¤®',
 'à¤\x89à¤ªà¤¯à¥\x8bà¤\x97',
 'à¤\x8aà¤\x82à¤\x9aà¤¾à¤\x88',
 'à¤\x8f',
 'à¤\x8fà¤¸à¤µà¥\x80à¤\x9cà¥\x80',
 'à¤\x94à¤°',
 'à¤\x95à¤°à¤¨à¤¾',
 'à¤\x95à¤°à¥\x87à¤\x82',
 'à¤\x95à¤¾',
 'à¤\x95à¤¾à¤\x9fà¥\x87à¤\x82',
 'à¤\x95à¤¾à¤°à¥\x8dà¤¯',
 'à¤\x95à¤¾à¤²',
 'à¤\x95à¤¿à¤¸à¥\x8dà¤®',
 'à¤\x95à¤à¥\x80',
 'à¤\x95à¥\x80',
 'à¤\x95à¥\x83à¤ªà¤¯à¤¾',
 'à¤\x95à¥\x87',
 'à¤\x95à¥\x88à¤²à¥\x87à¤£à¥\x8dà¤¡à¤°',
 'à¤\x95à¥\x89à¤¨à¥\x8dà¤«à¤¼à¤¿à¤\x97à¤°à¥\x87à¤¶à¤¨',
 'à¤\x95à¥\x8b',
 'à¤\x95à¥\x8bà¤¶à¤¿à¤¶',
 'à¤\x95à¥\x8dà¤²à¤¾à¤¸à¤¿à¤\x95',
 'à¤\x95à¥\x8dà¤²à¤¿à¤\x95',
 'à¤\x95à¥\x8dà¤µà¤¾à¤\x87à¤²',
 'à¤\x95à¥\

In [ ]:
hin_exclude = ['£',
 '¥',
 '§',
 '¨',
 '¨éé',
 '¨निश्चय',
 '©',
 '°',
 '°°',
 '°से',
 '°सेल्सियस',
 '²',
 '³',
 '´éšééç',
 'º',
 'º।',
 '¼',
 '¼½',
 '½',
 '½éþ।',
 '½θώ।',
 '½ιώ',
 '½ιώ।',
 '¾',
 '×',
 'à¤',
 'à¤\x86à¤\x87à¤\x95à¥\x89à¤¨à¥\x8dà¤¸',
 'à¤\x86à¤\x95à¤¾à¤°',
 'à¤\x86à¤«',
 'à¤\x86à¤°à¥\x87à¤\x96à¤¿à¤¤',
 'à¤\x86à¤°à¥\x8dà¤\x97à¥\x81à¤®à¥\x87à¤\x82à¤\x9f',
 'à¤\x86à¤µà¤¾à¤\x9c',
 'à¤\x87à¤\x82à¤¤à¤\x9cà¤¾à¤°',
 'à¤\x87à¤¸',
 'à¤\x89',
 'à¤\x89à¤¤à¥\x8dà¤¤à¤®',
 'à¤\x89à¤ªà¤¯à¥\x8bà¤\x97',
 'à¤\x8aà¤\x82à¤\x9aà¤¾à¤\x88',
 'à¤\x8f',
 'à¤\x8fà¤¸à¤µà¥\x80à¤\x9cà¥\x80',
 'à¤\x94à¤°',
 'à¤\x95à¤°à¤¨à¤¾',
 'à¤\x95à¤°à¥\x87à¤\x82',
 'à¤\x95à¤¾',
 'à¤\x95à¤¾à¤\x9fà¥\x87à¤\x82',
 'à¤\x95à¤¾à¤°à¥\x8dà¤¯',
 'à¤\x95à¤¾à¤²',
 'à¤\x95à¤¿à¤¯à¤¾',
 'à¤\x95à¤¿à¤¸à¥\x8dà¤®',
 'à¤\x95à¤à¥\x80',
 'à¤\x95à¥\x80',
 'à¤\x95à¥\x83à¤ªà¤¯à¤¾',
 'à¤\x95à¥\x87',
 'à¤\x95à¥\x88à¤²à¥\x87à¤£à¥\x8dà¤¡à¤°',
 'à¤\x95à¥\x89à¤¨à¥\x8dà¤«à¤¼à¤¿à¤\x97à¤°à¥\x87à¤¶à¤¨',
 'à¤\x95à¥\x8b',
 'à¤\x95à¥\x8bà¤¶à¤¿à¤¶',
 'à¤\x95à¥\x8dà¤²à¤¾à¤¸à¤¿à¤\x95',
 'à¤\x95à¥\x8dà¤²à¤¿à¤\x95',
 'à¤\x95à¥\x8dà¤µà¤¾à¤\x87à¤²',
 'à¤\x95à¥\x8dà¤·à¥\x87à¤¤à¥\x8dà¤°',
 'à¤\x96à¥\x87à¤²',
 'à¤\x96à¥\x8bà¤²à¥\x87à¤\x82',
 'à¤\x97à¤£à¤¨à¤¾',
 'à¤\x97à¤¯à¤¾',
 'à¤\x97à¥\x81à¤£à¥\x8bà¤\x82',
 'à¤\x9aà¤¯à¤¨',
 'à¤\x9aà¤¾à¤²à¥\x82',
 'à¤\x9aà¥\x81à¤¨à¤¤à¤¾',
 'à¤\x9aà¥\x81à¤¨à¥\x87à¤\x82',
 'à¤\x9aà¥\x8cà¤¡à¤¼à¤¾à¤\x88',
 'à¤\x9bà¥\x81à¤\x9fà¥\x8dà¤\x9fà¥\x80',
 'à¤\x9bà¥\x8bà¤¡à¤¼à¤¨à¤¾',
 'à¤\x9cà¤\x97à¤¹',
 'à¤\x9cà¤¾',
 'à¤\x9cà¤¾à¤\x82à¤\x9a',
 'à¤\x9cà¤¾à¤¨à¤¾',
 'à¤\x9cà¤¿à¤¨à¤¸à¥\x87',
 'à¤\x9cà¤¿à¤¸à¤¸à¥\x87',
 'à¤\x9cà¥\x81à¤¡à¤¼à¤¨à¤¾',
 'à¤\x9cà¥\x8bà¤¡à¤¼à¤¨à¥\x87',
 'à¤\x9cà¥\x8bà¤¡à¤¼à¥\x87à¤\x82',
 'à¤\x9cà¥\x8dà¤\x9eà¤¾à¤¤',
 'à¤\x9fà¥\x87à¤¬à¤²à¤\x83',
 'à¤¡à¤¬à¤²',
 'à¤¡à¤¾à¤\x9fà¤¾à¤¬à¥\x87à¤¸',
 'à¤¤à¤¤à¥\x8dà¤µà¥\x8bà¤\x82',
 'à¤¤à¤¥à¤¾',
 'à¤¤à¤¾à¤²à¤¿à¤\x95à¤¾',
 'à¤¦à¤¶à¤®à¤²à¤µ',
 'à¤¦à¤¿à¤\x96à¤¾à¤\x8fà¤\x81',
 'à¤¦à¥\x82à¤°à¥\x80',
 'à¤¦à¥\x87à¤\x96à¥\x87à¤\x82',
 'à¤§à¥\x8dà¤°à¥\x81à¤µ',
 'à¤¨',
 'à¤¨à¤\x95à¤¼à¤²',
 'à¤¨à¤¯à¤¾',
 'à¤¨à¤¹à¥\x80à¤\x82',
 'à¤¨à¤¾à¤®',
 'à¤¨à¤¾à¤®à¤\x83',
 'à¤¨à¥\x81à¤ªà¤¾à¤¤',
 'à¤ªà¤\x9fà¥\x8dà¤\x9fà¥\x80',
 'à¤ªà¤°',
 'à¤ªà¤¾à¤¸',
 'à¤ªà¥\x80à¤\x8fà¤\x95à¥\x8dà¤¸',
 'à¤ªà¥\x82',
 'à¤ªà¥\x8bà¤\x82à¤\x9bà¤¨à¤¾',
 'à¤ªà¥\x8bà¤°à¥\x8dà¤\x9f',
 'à¤ªà¥\x8dà¤°à¤\x97à¤¤à¤¿',
 'à¤ªà¥\x8dà¤°à¤¤à¥\x80à¤\x95à¥\x8dà¤·à¤¾',
 'à¤ªà¥\x8dà¤°à¤®à¤¾à¤£à¤ªà¤¤à¥\x8dà¤°à¥\x8bà¤\x82',
 'à¤ªà¥\x8dà¤°à¤¯à¥\x8bà¤\x97',
 'à¤ªà¥\x8dà¤°à¤µà¤¿à¤·à¥\x8dà¤\x9f',
 'à¤ªà¥\x8dà¤°à¤à¤¾à¤µ',
 'à¤ªà¥\x8dà¤°à¥\x8bà¤\x9fà¥\x8bà¤\x95à¥\x89à¤²',
 'à¤ªà¥\x8dà¤°à¥\x8bà¤«à¤¼à¤¾à¤\x87à¤²',
 'à¤«à¤¼à¤¾à¤\x87à¤²',
 'à¤«à¤¼à¤¾à¤\x87à¤²à¤¨à¤¾à¤®',
 'à¤«à¤¼à¥\x89à¤°à¥\x8dà¤®à¥\x87à¤\x9f',
 'à¤«à¤¼à¥\x8bà¤²à¥\x8dà¤¡à¤°',
 'à¤«à¤¿à¤°',
 'à¤«à¥\x87à¤®',
 'à¤¬à¤¨à¤¾à¤¯à¤¾',
 'à¤¬à¤¹à¤¨à¥\x8bà¤\x82',
 'à¤¬à¥\x8bà¤²à¤¨à¥\x87',
 'à¤¬à¥\x8dà¤°à¤¶',
 'à¤¬à¥\x8dà¤°à¤¾à¤\x89à¤\x9cà¤¼à¤°',
 'à¤®à¤¿à¤\x9fà¤¾à¤\x8fà¤\x81',
 'à¤®à¤¿à¤¨à¤\x9f',
 'à¤®à¥\x80à¤¡à¤¿à¤¯à¤¾',
 'à¤®à¥\x87à¤\x82',
 'à¤®à¥\x8dà¤²',
 'à¤°à¤\x82à¤\x97',
 'à¤°à¤¨',
 'à¤°à¤¹à¤¾',
 'à¤°à¤¿à¤\x95à¥\x8dà¤¤',
 'à¤°à¥\x80à¤\x95à¤®à¥\x8dà¤ªà¤¾à¤\x87à¤²',
 'à¤²à¤¿à¤\x8f',
 'à¤²à¤¿à¤\x96à¥\x87à¤\x82',
 'à¤µà¤¾à¤²à¤¾',
 'à¤µà¤¾à¤²à¤¾à¤ªà¤¾à¤¨à¥\x87',
 'à¤µà¤¿à¤\x95à¤²à¥\x8dà¤ª',
 'à¤µà¤¿à¤¨à¥\x8dà¤¯à¤¾à¤¸',
 'à¤µà¥\x88à¤\x95à¤²à¥\x8dà¤ªà¤¿à¤\x95',
 'à¤µà¥\x88à¤§',
 'à¤¶à¥\x8b',
 'à¤¸à¤\x82à¤\x96à¥\x8dà¤¯à¤¾',
 'à¤¸à¤\x82à¤¦à¥\x87à¤¶à¥\x8bà¤\x82',
 'à¤¸à¤\x82à¤ªà¤¾à¤¦à¤\x95',
 'à¤¸à¤\x82à¤ªà¤¾à¤¦à¤¿à¤¤',
 'à¤¸à¤¬à¥\x8dà¤¸à¤\x95à¥\x8dà¤°à¤¿à¤ªà¥\x8dà¤¶à¤¨',
 'à¤¸à¤¹à¥\x87à¤\x9cà¤¾',
 'à¤¸à¤¹à¥\x87à¤\x9cà¥\x87à¤\x82',
 'à¤¸à¤¾à¤«',
 'à¤¸à¤¿à¤\x82à¤¥à¥\x87à¤¸à¤¾à¤\x87à¤\x9cà¤°',
 'à¤¸à¥\x82à¤\x9aà¤¨à¤¾',
 'à¤¸à¥\x82à¤\x9aà¥\x80',
 'à¤¸à¥\x87',
 'à¤¸à¥\x87à¤\x9f',
 'à¤¸à¥\x8dà¤\x95à¥\x87à¤²',
 'à¤¸à¥\x8dà¤\x95à¥\x87à¤²à¥\x8dà¤¸',
 'à¤¸à¥\x8dà¤\x95à¥\x88à¤¨',
 'à¤¸à¥\x8dà¤\x95à¥\x8dà¤°à¤¿à¤ªà¥\x8dà¤\x9f',
 'à¤¸à¥\x8dà¤\x95à¥\x8dà¤°à¥\x89à¤²à¤¬à¥\x88à¤\x95',
 'à¤¸à¥\x8dà¤\x95à¥\x8dà¤µà¥\x87à¤°à¤¿à¤\x95',
 'à¤¸à¥\x8dà¤\x9fà¤¾à¤ª',
 'à¤¸à¥\x8dà¤\x9fà¥\x80à¤\x88à¤°à¥\x80à¤¨',
 'à¤¸à¥\x8dà¤\x9fà¥\x87à¤\x95',
 'à¤¸à¥\x8dà¤\x9fà¥\x8dà¤°à¤¿à¤\x82à¤\x97',
 'à¤¸à¥\x8dà¤\x9fà¥\x8dà¤°à¤¿à¤\x82à¤\x97à¤\x83',
 'à¤¸à¥\x8dà¤\x9fà¥\x8dà¤°à¥\x80à¤®',
 'à¤¸à¥\x8dà¤¤à¤®à¥\x8dà¤',
 'à¤¸à¥\x8dà¤¤à¤°',
 'à¤¸à¥\x8dà¤¤à¤°à¤\x83',
 'à¤¸à¥\x8dà¤¤à¥\x8dà¤°à¥\x80',
 'à¤¸à¥\x8dà¤¥à¤¾à¤¨',
 'à¤¸à¥\x8dà¤¥à¤¾à¤¨à¤\x83',
 'à¤¸à¥\x8dà¤¥à¤¾à¤¨à¥\x80à¤¯',
 'à¤¸à¥\x8dà¤¥à¤¿à¤¤à¤¿',
 'à¤¸à¥\x8dà¤¥à¤¿à¤¤à¤¿à¤\x83',
 'à¤¸à¥\x8dà¤¥à¤¿à¤°à¤¾à¤\x82à¤\x95',
 'à¤¸à¥\x8dà¤¨à¥\x87à¤ª',
 'à¤¸à¥\x8dà¤ªà¤¿à¤¨',
 'à¤¸à¥\x8dà¤ªà¥\x80à¤\x9a',
 'à¤¸à¥\x8dà¤ªà¥\x87à¤\x95à¥\x8dà¤\x9fà¤¾à¤\x87à¤ªà¤\x83',
 'à¤¸à¥\x8dà¤ªà¥\x87à¤¸à¥\x80',
 'à¤¸à¥\x8dà¤ªà¥\x87à¤¸à¥\x87à¤¸',
 'à¤¸à¥\x8dà¤ªà¥\x87à¤¸à¥\x8dà¤¡',
 'à¤¸à¥\x8dà¤ªà¥\x88à¤®',
 'à¤¸à¥\x8dà¤ªà¥\x88à¤®à¤¹à¥\x88à¤®',
 'à¤¸à¥\x8dà¤®à¤¾à¤°à¥\x8dà¤\x9f',
 'à¤¸à¥\x8dà¤®à¥\x80à¤¯à¤°à¥\x80',
 'à¤¸à¥\x8dà¤°à¥\x8bà¤¤',
 'à¤¸à¥\x8dà¤²à¤¾à¤\x87à¤¡',
 'à¤¸à¥\x8dà¤²à¤¾à¤\x87à¤¡à¤°',
 'à¤¸à¥\x8dà¤²à¤¾à¤\x87à¤¡à¤°à¥\x8bà¤\x82',
 'à¤¸à¥\x8dà¤²à¤¾à¤\x87à¤¡à¤¶à¥\x8b',
 'à¤¸à¥\x8dà¤²à¤¿à¤\x95',
 'à¤¸à¥\x8dà¤²à¥\x89à¤\x9f',
 'à¤¸à¥\x8dà¤µà¤\x9aà¤²à¤¿à¤¤',
 'à¤¸à¥\x8dà¤µà¤\x9aà¤¾à¤²à¤¿à¤¤',
 'à¤¸à¥\x8dà¤µà¤¤à¤\x83',
 'à¤¸à¥\x8dà¤µà¤¤à¤\x83à¤\x95à¥\x8dà¤·à¥\x87à¤¤à¥\x8dà¤°',
 'à¤¸à¥\x8dà¤µà¤¤à¤\x83à¤¸à¤\x82à¤\x96à¥\x8dà¤¯à¤¾',
 'à¤¸à¥\x8dà¤µà¤¯à¤\x82',
 'à¤¹à¤\x9fà¤¾à¤\x8fà¤\x81',
 'à¤¹à¤®à¥\x87à¤¶à¤¾',
 'à¤¹à¤²à¥\x8dà¤\x95à¤¾',
 'à¤¹à¤¸à¥\x8dà¤¤à¤¾à¤\x82à¤¤à¤°à¤£à¤ªà¤°',
 'à¤¹à¤¸à¥\x8dà¤¤à¤¾à¤\x95à¥\x8dà¤·à¤°',
 'à¤¹à¤¾à¤°à¤¾à¤\x83',
 'à¤¹à¤¾à¤°à¥\x8dà¤¡',
 'à¤¹à¤¾à¤²',
 'à¤¹à¤¾à¤²à¤¿à¤¯à¤¾',
 'à¤¹à¤¾à¤¶à¤¿à¤¯à¤¾',
 'à¤¹à¤¿à¤¸à¥\x8dà¤\x9fà¥\x8bà¤\x97à¥\x8dà¤°à¤¾à¤®',
 'à¤¹à¥\x80',
 'à¤¹à¥\x80à¤\x9fà¤¿à¤\x82à¤\x97',
 'à¤¹à¥\x87à¤\x95à¥\x8dà¤¸',
 'à¤¹à¥\x87à¤¡à¤°',
 'à¤¹à¥\x87à¤¤à¥\x81',
 'à¤¹à¥\x88',
 'à¤¹à¥\x88à¤\x83',
 'à¤¹à¥\x88à¤¡à¤¿à¤«à¤¼à¤¿à¤\x95à¥\x8dà¤¸',
 'à¤¹à¥\x88à¤²à¥\x8bà¤\x9cà¤¨',
 'à¤¹à¥\x88à¤²à¥\x8bà¤\x9cà¤¨à¥\x8dà¤¸',
 'à¤¹à¥\x89à¤\x9fà¤¸à¤¿à¤\x82à¤\x95',
 'à¤¹à¥\x89à¤²à¥\x8dà¤¸à¥\x8dà¤\x9fà¥\x87à¤\x82à¤\x9fà¤°',
 'à¤¹à¥\x8bà¤®',
 'à¤¹à¥\x8bà¤¸à¥\x8dà¤\x9f',
 'à¤¹à¥\x8bà¤¸à¥\x8dà¤\x9fà¤\x83',
 'à¤à¤\x82à¤¡à¤¾à¤°',
 'à¤à¤°à¤¾à¥\x80',
 'à¤à¤¾à¤\x87à¤¯à¥\x8bà¤\x82',
 'à¤à¤¾à¤·à¤¾',
 'à¤à¤¿à¤¨à¥\x8dà¤¨',
 'à¤à¥\x82à¤®à¤¿à¤\x95à¤¾',
 'à¤à¥\x87à¤\x9cà¤¨à¥\x87',
 'à¤à¥\x87à¤\x9cà¥\x87à¤\x82',
 'à¤à¥\x8cà¤\x97à¥\x8bà¤²à¤¿à¤\x95',
 'à¤à¥\x8cà¤¤à¤¿à¤\x95',
 'à¥¨',
 'â\x80\x9cसीता',
 'â°',
 'â˛',
 'æ',
 'çण',
 'é',
 'éä',
 'éäž¤éúसे',
 'éčžú',
 'éš',
 'éई',
 'ð',
 'ö',
 '÷',
 'ú',
 'úतरीकों',
 'úदाहिने',
 'úहोता।',
 'ćśé',
 'ćकी',
 'č।',
 'ľ',
 'ő',
 'őęžúşééĺ',
 'őĺňşé',
 'őĺş',
 'őň',
 'śकहा।',
 'ş',
 'şé',
 'şéě´éşé।',
 'şéň',
 'ţभी',
 'ţसुनकर',
 'ţ।',
 'ů',
 'žúéš',
 'ˇ',
 '˙˙˙',
 '˙˙˙बचपन',
 '˙˙˙मैं',
 '˜हां™',
 '˝',
 '˝éţ',
 '˝éţ।',
 '˝čţ।',
 '˝ţ¨éĺ',
 '˝ţéä।',
 '˝ţéĺ।',
 '́लीड',
 '́सारा',
 'α',
 'β',
 'βसंयोजी',
 'γ',
 'δ',
 'θ',
 'κ',
 'λ',
 'μ',
 'ν',
 'π',
 'π∕',
 'τ',
 'φ',
 'ω',
 'андрей',
 'иии',
 'ииииहाय',
 'иииमैं',
 'ииउस',
 'ииदेवसिंह',
 'ииहमारे',
 'с',
 'сकाटेगीт',
 'т',
 'черепанов',
 'الإسلام',
 'الإسلام',
 '’।',
 '’’',
 '’’।',
 '‚',
 '„',
 '‡कए',
 '‡कविश्वविद्यालय',
 '‡लया',
 '‡हन्दी',
 '…',
 '…एक',
 '…।',
 '………',
 '……………',
 '………………से',
 '′',
 '⁰',
 '⁻¹⁰⁰',
 '₹',
 '™',
 '∨',
 '♪',
 '♫',
 '♫इट्स',
 '\uf00f',
 '�',
 '��ाउनलोड']

In [ ]:
'''print(len(hin_exclude))
for p in hin_exclude:
    df1['hin'] = df1['hin'].str.replace(p,'-1')'''

"print(len(hin_exclude))\nfor p in hin_exclude:\n    df1['hin'] = df1['hin'].str.replace(p,'-1')"

In [ ]:
def f2(string):
  for i in string.split():
    for j in i:
      if j in hin_exclude:
        return '-1'
  return string

In [ ]:
df1['hin'] = df1['hin'].apply(lambda x: f2(x), )

In [ ]:
df1.shape

(1056824, 4)

In [ ]:
l=[]
for i in df1.index:
  if df1['hin'][i]=='-1':
    l.append(i)
df1.drop(l,inplace=True)

In [ ]:
df1.shape

(1055993, 4)

In [ ]:
df1.head(50)

,eng,hin,length_eng_sentence,length_hin_sentence
1485176,sita kalyana marriage of sita is a favourite t...,START_ âसीता कल्याणâ सीता का विवाह अनेक लो...,18,11
1398169,prem chand s mother passed away after an illne...,START_ ˙˙˙ छः महीने कीबामारी के बाद प्रेमचन्द ...,12,13
1398143,among all the sweet memories of childhood this...,START_ ˙˙˙बचपन की मीठी स्मृतीयों में गिल्ली ही...,11,10
1398893,i do not desire even to live in peace,START_ ˙˙˙मैं शान्ति से बैठना भी नहीं चाहता। _END,9,7
1417047,but do i want to preach any sermon,START_ иии परन्तु यह तो उपदेश देना होगा _END,8,7
1307807,it is man s nature to do good for all selves a...,START_ иии भलाई करना मनुष्य का स्वभाव है क्यों...,13,12
1417283,ah i had committed a mistake by writing a lett...,START_ ииииहाय मैंने चिट्ठी लिखकर बहुत बुरा कि...,12,7
1417055,why should i not adopt the same method,START_ иииमैं भी क्यों न कुछ ऐसा ही ढंग अपनाऊँ...,8,9
1417044,can we take something from that age for our use,START_ ииउस युग से क्या आज के लिए कुछ ग्रहण कि...,10,13
1417297,devi singh has brought her here,START_ ииदेवसिंह लिवा लाया है। _END,6,4


In [ ]:
df1

,eng,hin,length_eng_sentence,length_hin_sentence
1485176,sita kalyana marriage of sita is a favourite t...,START_ âसीता कल्याणâ सीता का विवाह अनेक लो...,18,11
1398169,prem chand s mother passed away after an illne...,START_ ˙˙˙ छः महीने कीबामारी के बाद प्रेमचन्द ...,12,13
1398143,among all the sweet memories of childhood this...,START_ ˙˙˙बचपन की मीठी स्मृतीयों में गिल्ली ही...,11,10
1398893,i do not desire even to live in peace,START_ ˙˙˙मैं शान्ति से बैठना भी नहीं चाहता। _END,9,7
1417047,but do i want to preach any sermon,START_ иии परन्तु यह तो उपदेश देना होगा _END,8,7
...,...,...,...,...
1072670,denominator,START_ होगा हर में _END,1,3
921187,hogarth,START_ होगार्थ _END,1,1
1422440,if perchance he does he will cure himself with...,START_ होगी भी तो वह उसे पंच महाभूतों की मदद स...,14,13
1265612,hogenakkal falls,START_ होगेन्नाकल _END,2,1


# Preprocessing Done, Encoder decoder model

In [ ]:
df1 = shuffle(df1)

In [ ]:
df1 = df1.sort_index(axis = 0) 

In [ ]:
df1 = df1[:30000]

In [ ]:
df1.shape

(30000, 4)

In [ ]:
df1['length_eng_sentence']=df1['eng'].apply(lambda x:len(x.split(" ")))
df1['length_hin_sentence']=df1['hin'].apply(lambda x:len(x.split(" ")))


In [ ]:
### Get English and Hindi Vocabulary
all_hindi_words=set()
for hin in df1['hin']:
    for word in hin.split():
        if word not in all_hindi_words:
            all_hindi_words.add(word)

all_eng_words=set()
for eng in df1['eng']:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)
print(len(all_eng_words))
print(len(all_hindi_words))

2497
2807


In [ ]:
max_length_src=max(df1['length_eng_sentence'])
max_length_tar=max(df1['length_hin_sentence'])

In [ ]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_hindi_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_hindi_words)
num_encoder_tokens, num_decoder_tokens

(2497, 2807)

In [ ]:
num_decoder_tokens += 1 #for zero padding

In [ ]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])


In [ ]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

Split the data into train and test

In [ ]:
X, y = df1['eng'], df1['hin']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=42)
X_train.shape, X_test.shape

((24000,), (6000,))

In [ ]:
X_train.to_pickle('/content/Drive/My Drive/Application of ML in industries/NMT corpus IITB/X_train')
X_test.to_pickle('/content/Drive/My Drive/Application of ML in industries/NMT corpus IITB/X_test')

In [ ]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

Encoder-Decoder Architecture

In [ ]:
latent_dim=10

In [ ]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [ ]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 10)     24970       input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 10)     28080       input_2[0][0]                    
______________________________________________________________________________________________

In [ ]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 32
epochs = 2

In [ ]:
val_samples

6000

In [ ]:
batch_size


32

In [ ]:
val_samples//batch_size

187

In [ ]:
df1.head(45)

,eng,hin,length_eng_sentence,length_hin_sentence
0,give your application an accessibility workout,START_ अपने अनुप्रयोग को पहुंचनीयता व्यायाम का...,6,10
1,accerciser accessibility explorer,START_ एक्सेर्साइसर पहुंचनीयता अन्वेषक _END,3,5
2,the default plugin layout for the bottom panel,START_ निचले पटल के लिए डिफोल्ट प्लगइन खाका _END,8,9
3,the default plugin layout for the top panel,START_ ऊपरी पटल के लिए डिफोल्ट प्लगइन खाका _END,8,9
4,a list of plugins that are disabled by default,START_ उन प्लगइनों की सूची जिन्हें डिफोल्ट रूप...,9,14
5,highlight duration,START_ अवधि को हाइलाइट रकें _END,2,6
6,the duration of the highlight box when selecti...,START_ पहुंचनीय आसंधि नोड को चुनते समय हाइलाइट...,10,12
7,highlight border color,START_ सीमांत बोर्डर के रंग को हाइलाइट करें _END,3,9
8,the color and opacity of the highlight border,START_ हाइलाइट किए गए सीमांत का रंग और अपारदर्...,8,10
9,highlight fill color,START_ भराई के रंग को हाइलाइट करें _END,3,8


In [ ]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)

Epoch 1/2


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


 43/750 [>.............................] - ETA: 11s - loss: 1.6358

InvalidArgumentError: ignored

In [ ]:
df1.shape

In [ ]:
df1.index

In [ ]:
df1 = df1.sort_index(axis = 0) 

In [ ]:
df1.head(50)

In [ ]:
decoder_input_data[i, t]